#### Data is taken from Five-Thirty-Eight
(https://github.com/fivethirtyeight/guns-data)


In [1]:
import csv 

f = open("guns.csv", "r")
gun_data = csv.reader(f)
gun_data = list(gun_data)

### Check that we read in data

In [2]:
gun_data[:1]

[['',
  'year',
  'month',
  'intent',
  'police',
  'sex',
  'age',
  'race',
  'hispanic',
  'place',
  'education']]

In [3]:
headers = gun_data.pop(0)


In [4]:
import collections

year_col = 1
year_counts = collections.Counter([death[year_col] for death in gun_data])
year_counts

Counter({'2012': 33563, '2013': 33636, '2014': 33599})

### Very little change in number of gun deaths over 3 years.  Gun deaths from 2012 to 2014 ranged from 33563 to 33599 deaths per year.

##### Next, use the datetime module to convert the month and year columns into an easily analyzable datetime object column

In [5]:
from datetime import datetime

year_c = 1
month_c = 2
date = 1

dates = [datetime(int(death[year_c]),
                  int(death[month_c]),
                  day=date) for death in gun_data]

dates[:5]

[datetime.datetime(2012, 1, 1, 0, 0),
 datetime.datetime(2012, 1, 1, 0, 0),
 datetime.datetime(2012, 1, 1, 0, 0),
 datetime.datetime(2012, 2, 1, 0, 0),
 datetime.datetime(2012, 2, 1, 0, 0)]

In [6]:
date_counts = collections.Counter([date for date in dates])

date_counts

Counter({datetime.datetime(2012, 1, 1, 0, 0): 2758,
         datetime.datetime(2012, 2, 1, 0, 0): 2357,
         datetime.datetime(2012, 3, 1, 0, 0): 2743,
         datetime.datetime(2012, 4, 1, 0, 0): 2795,
         datetime.datetime(2012, 5, 1, 0, 0): 2999,
         datetime.datetime(2012, 6, 1, 0, 0): 2826,
         datetime.datetime(2012, 7, 1, 0, 0): 3026,
         datetime.datetime(2012, 8, 1, 0, 0): 2954,
         datetime.datetime(2012, 9, 1, 0, 0): 2852,
         datetime.datetime(2012, 10, 1, 0, 0): 2733,
         datetime.datetime(2012, 11, 1, 0, 0): 2729,
         datetime.datetime(2012, 12, 1, 0, 0): 2791,
         datetime.datetime(2013, 1, 1, 0, 0): 2864,
         datetime.datetime(2013, 2, 1, 0, 0): 2375,
         datetime.datetime(2013, 3, 1, 0, 0): 2862,
         datetime.datetime(2013, 4, 1, 0, 0): 2798,
         datetime.datetime(2013, 5, 1, 0, 0): 2806,
         datetime.datetime(2013, 6, 1, 0, 0): 2920,
         datetime.datetime(2013, 7, 1, 0, 0): 3079,
         

In [7]:
sex_col = 5

sex_counts = collections.Counter([death[sex_col] for death in gun_data])

sex_counts

Counter({'F': 14449, 'M': 86349})

In [8]:
race_col = 7

race_deaths = collections.Counter([death[race_col] for death in gun_data])

race_deaths

Counter({'Asian/Pacific Islander': 1326,
         'Black': 23296,
         'Hispanic': 9022,
         'Native American/Native Alaskan': 917,
         'White': 66237})

### Males were 6 times as likely to be a victim of a gun related death than Females from 2012-2014.
###### How does this correspond with suicide rates?
### Whites were 3 times as likely to be involved in a gun death than blacks.
##### How does this correspond with the proportion of whites/blacks in the US?


# Use 2010 US Census data to get a more accurate picture

#### Read in our data (credit to www.census.gov)

In [9]:
import pandas as pd

census_f = "census.csv"
census = pd.read_csv(census_f)


### Let's use census data to get rates of deaths per 100k people per race.

##### The surveys have different names for each race.  Let's map the races together.

In [10]:
#Map of `gun_data => census`

race_map = {
    "Asian/Pacific Islander": ["Race Alone - Asian",
                               "Race Alone - Native Hawaiian and Other Pacific Islander"],
    "Black": ["Race Alone - Black or African American"],
    "Hispanic": ["Race Alone - Hispanic"],
    "Native American/Native Alaskan": ["Race Alone - American Indian and Alaska Native"],
    "White": ["Race Alone - White"]
}
race_map

{'Asian/Pacific Islander': ['Race Alone - Asian',
  'Race Alone - Native Hawaiian and Other Pacific Islander'],
 'Black': ['Race Alone - Black or African American'],
 'Hispanic': ['Race Alone - Hispanic'],
 'Native American/Native Alaskan': ['Race Alone - American Indian and Alaska Native'],
 'White': ['Race Alone - White']}

### Once we map the two datasets together.  We can calculate the deaths per 100k of population.  

In [11]:
def convert_deaths_to_per100k(race_deaths,census, race_map):
    deaths_per_100k = {}
    for race, death in race_deaths.items():
        # Get the names of the race in the census that map to the gun data names
        census_names = race_map[race]
        census_pops = [census.iloc[0][name] for name in census_names]

        #Total race population from the census data
        race_population = sum(census_pops)


        deaths_per_hundredk = (death * 100000) / race_population 
        deaths_per_hundredk = int(deaths_per_hundredk)
        deaths_per_100k[race] = deaths_per_hundredk
    return deaths_per_100k
    
race_per_hundredk = convert_deaths_to_per100k(race_deaths,
                                              census,
                                              race_map)
race_per_hundredk
    
    

{'Asian/Pacific Islander': 8,
 'Black': 57,
 'Hispanic': 20,
 'Native American/Native Alaskan': 24,
 'White': 33}

### Deaths per 100k:
##### Black deaths per 100k are nearly double the rate of White deaths 
##### Our data spans 2012 to 2014, and includes all deaths (i.e. Suicide, Homicide, Accidental, Undetermined, etc.)

### Look at types of deaths in our gun data

In [12]:
type_col = 3

death_type = collections.Counter([death[type_col] for death in gun_data])
death_type

Counter({'Accidental': 1639,
         'Homicide': 35176,
         'NA': 1,
         'Suicide': 63175,
         'Undetermined': 807})

### Examine *homicide* gun deaths per 100k broken down by race

##### First, redo our gun death counts

In [13]:

homicide_deaths = collections.Counter([death[race_col] for death in gun_data if death[type_col] == "Homicide"])
homicide_deaths

Counter({'Asian/Pacific Islander': 559,
         'Black': 19510,
         'Hispanic': 5634,
         'Native American/Native Alaskan': 326,
         'White': 9147})

##### There were 19510 black deaths by homicide from 2012 to 2014 in the United States.

In [14]:
race_homicide_per_hundredk = convert_deaths_to_per100k(homicide_deaths,
                                                       census,
                                                       race_map)
race_homicide_per_hundredk

{'Asian/Pacific Islander': 3,
 'Black': 48,
 'Hispanic': 12,
 'Native American/Native Alaskan': 8,
 'White': 4}

##### There were 12 times as many Black deaths by gun [homicide] from 2012 to 2014 in the United States.